# Hyper-Parameter Tuning

This notebook contains hyperparameter tuning for the model. Learner module, FastAI, provides a convenient way to create and fine-tune convolutional neural network (CNN) models. vision.learner is a function that helps us to construct a learner object, which has the model architecture, data, training configuration, and other elements. We can specify a pre-trained model architecture and fine-tune it on the dataset. vision.learner supports a wide range of CNN architectures.

This notebook includes these following implementations:

1. Random Search optimization algorithm - Run 1
2. Random Search optimization algorithm - Run 2
3. Hyperparameter Optimization with Optuna's Successive Halving Pruner

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install optuna-integration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 2.5 MB/s eta 0:00:00


In [3]:
!pip install optuna lightgbm

In [4]:
!pip install --upgrade optuna

# Random Search optimization algorithm - Run 1

The optimization algorithm used below is Optuna's default algorithm, which is a Tree-structured Parzen Estimator (TPE) algorithm.

For n_trails = 10, the accuracy score and best hyper-parameter are as follows:

| Trail No. | Best Score | Architecture | Weight Decay | Epochs | Batch Size | Drop |
|-----------|------------|--------------|--------------|--------|------------|------|
| 0         | 0.9011     | ResNet34     | 0.00024      | 8      | 64         | 0.4  |
| 1         | 0.9413     | ResNet18     | 0.0090       | 15     | 64         | 0.2  |
| 2         | 0.9343     | ResNet18     | 0.0065       | 5      | 32         | 0.4  |
| 3         | 0.9080     | ResNet34     | 0.00062      | 5      | 32         | 0.4  |
| 4         | 0.9019     | ResNet34     | 0.00092      | 6      | 64         | 0.2  |
| 5         | 0.9527     | ResNet50     | 0.00005      | 7      | 64         | 0.4  |
| 6         | 0.9220     | ResNet34     | 0.00895      | 15     | 64         | 0.2  |
| 7         | 0.9404     | ResNet18     | 0.0035       | 11     | 64         | 0.4  |
| 8         | 0.9212     | ResNet18     | 0.0002       | 5      | 64         | 0.4  |
| 9         | 0.9203     | ResNet34     | 0.0007       | 13     | 32         | 0.4  |


**The best aacuracy score is 0.9527 with these hyperparameters:**
- Architecture: ResNet 50
- Weight Decay: 5.527e-5
- Epochs: 7
- Batch Size: 64
- Drop: 0.4

In [5]:
import optuna
from fastai.vision.all import *
from sklearn.model_selection import cross_val_score

# objective function for hyperparameter optimization
def objective(trial):
    path = Path('/kaggle/input/brain-tumor-mri-classification-dataset/Brain_Tumor_MRI_Image_Dataset/Training')
    
    # hyperparameters to optimize
    arch = trial.suggest_categorical('arch', ['resnet18', 'resnet34', 'resnet50'])
    wd = trial.suggest_loguniform('wd', 1e-6, 1e-2)
    epochs = trial.suggest_int('epochs', 5, 15)
    bs = trial.suggest_categorical('bs', [32, 64])
    drop = trial.suggest_categorical('drop', [0.2, 0.4])
    
    # DataBlock to prepare the data
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       get_items=get_image_files,
                       get_y=parent_label,
                       splitter=RandomSplitter(valid_pct=0.2, seed=42))
    
    dls = dblock.dataloaders(path, bs=bs) # Create DataLoaders
    
    # learner with specified architecture, metrics, weight decay, and data
    learn = vision_learner(dls, arch, metrics=accuracy, wd=wd)
    
    # Fine-tune the model
    learn.fine_tune(epochs, base_lr=0.001, cbs=[MixedPrecision()])
    
    return float(learn.validate()[1]) # validation accuracy of the trained model

# Optuna study object for hyperparameter optimization
study = optuna.create_study(direction="maximize")

# Optimize the objective function by running multiple trials
study.optimize(objective, n_trials=10)

trial = study.best_trial # best trial from the study

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-05-24 21:50:09,794] A new study created in memory with name: no-name-10ac8b0a-751b-4f7e-b013-29bab511d22f


model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,1.096213,0.387679,0.865149,00:14


epoch,train_loss,valid_loss,accuracy,time
0,0.568723,0.322058,0.892294,00:16
1,0.490151,0.291415,0.892294,00:15
2,0.417919,0.286482,0.898424,00:16
3,0.361479,0.278917,0.884413,00:16
4,0.310722,0.246992,0.907180,00:16
5,0.267015,0.246769,0.904553,00:16
6,0.226637,0.251126,0.898424,00:16
7,0.210725,0.248565,0.901926,00:16


[I 2024-05-24 21:52:42,748] Trial 0 finished with value: 0.9010508060455322 and parameters: {'arch': 'resnet34', 'wd': 0.00023340994424944704, 'epochs': 8, 'bs': 64, 'drop': 0.4}. Best is trial 0 with value: 0.9010508060455322.


...

[I 2024-05-24 22:11:41,417] Trial 8 finished with value: 0.9211909174919128 and parameters: {'arch': 'resnet18', 'wd': 0.00020852879448736512, 'epochs': 5, 'bs': 64, 'drop': 0.4}. Best is trial 5 with value: 0.9527145624160767.


epoch,train_loss,valid_loss,accuracy,time
0,0.910820,0.359393,0.868652,00:13


epoch,train_loss,valid_loss,accuracy,time
0,0.537367,0.309434,0.897548,00:17
1,0.480948,0.295927,0.887040,00:17
2,0.435401,0.249373,0.912434,00:17
3,0.376859,0.250839,0.903678,00:17
4,0.316053,0.253130,0.906305,00:17
5,0.278214,0.218644,0.918564,00:17
6,0.260835,0.218007,0.908056,00:17
7,0.231998,0.208978,0.921191,00:17
8,0.186753,0.204182,0.922067,00:17
9,0.184746,0.203614,0.917688,00:17


[I 2024-05-24 22:15:43,586] Trial 9 finished with value: 0.9203152656555176 and parameters: {'arch': 'resnet34', 'wd': 0.0007385132863360378, 'epochs': 13, 'bs': 32, 'drop': 0.4}. Best is trial 5 with value: 0.9527145624160767.


Accuracy: 0.9527145624160767
Best hyperparameters: {'arch': 'resnet50', 'wd': 5.527401639726816e-05, 'epochs': 7, 'bs': 64, 'drop': 0.4}


# Randomized search for hyperparameter optimization - Run 2


For n_trails = 10, the accuracy score and best hyper-parameter are as follows:

| Trial | Best Score | Architecture | Weight Decay | Epochs | Batch Size | Drop |
|-------|------------|--------------|--------------|--------|------------|------|
| 0     | 0.9177     | ResNet34     | 0.000248     | 6      | 32         | 0.2  |
| 1     | 0.9492     | ResNet50     | 0.002137     | 7      | 64         | 0.4  |
| 2     | 0.9518     | ResNet50     | 0.000004     | 8      | 64         | 0.2  |
| 3     | 0.9046     | ResNet34     | 0.000269     | 6      | 64         | 0.2  |
| 4     | 0.9378     | ResNet50     | 0.000058     | 6      | 32         | 0.2  |
| 5     | 0.9352     | ResNet18     | 0.000154     | 7      | 32         | 0.2  |
| 6     | 0.9063     | ResNet34     | 0.000006     | 5      | 64         | 0.4  |
| 7     | 0.9553     | ResNet50     | 0.000004     | 13     | 64         | 0.2  |
| 8     | 0.9238     | ResNet34     | 0.000824     | 13     | 64         | 0.4  |
| 9     | 0.9361     | ResNet18     | 0.000018     | 8      | 32         | 0.2  |


**The best aacuracy score is 0.9553 with these hyperparameters:**
- Architecture: ResNet 50
- Weight Decay: 4e-6
- Epochs: 5
- Batch Size: 64
- Drop: 0.2

In [6]:
import optuna
from fastai.vision.all import *
from sklearn.model_selection import cross_val_score
from optuna.samplers import TPESampler

def objective(trial):
    path = Path('/kaggle/input/brain-tumor-mri-classification-dataset/Brain_Tumor_MRI_Image_Dataset/Training')
    
    arch = trial.suggest_categorical('arch', ['resnet18', 'resnet34', 'resnet50'])
    wd = trial.suggest_loguniform('wd', 1e-6, 1e-2)
    epochs = trial.suggest_int('epochs', 5, 15)
    bs = trial.suggest_categorical('bs', [32, 64])
    drop = trial.suggest_categorical('drop', [0.2, 0.4])

    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       get_items=get_image_files,
                       get_y=parent_label,
                       splitter=RandomSplitter(valid_pct=0.2, seed=42))
    
    dls = dblock.dataloaders(path, bs=bs)

    learn = vision_learner(dls, arch, metrics=accuracy, wd=wd)
    learn.fine_tune(epochs, base_lr=0.001, cbs=[MixedPrecision()])

    return float(learn.validate()[1])

sampler = TPESampler(seed=42)  # Initialize TPESampler for random search
study = optuna.create_study(direction="maximize", sampler=sampler)  # Use TPESampler for random search
study.optimize(objective, n_trials=10)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-05-24 22:15:43,603] A new study created in memory with name: no-name-b9e79e9e-2be7-4f61-82ff-92e3b89629f2


epoch,train_loss,valid_loss,accuracy,time
0,0.861309,0.381287,0.870403,00:13


epoch,train_loss,valid_loss,accuracy,time
0,0.560308,0.326959,0.875657,00:17
1,0.489167,0.303019,0.886165,00:17
2,0.382597,0.276688,0.898424,00:17
3,0.365501,0.247717,0.904553,00:18
4,0.277556,0.245371,0.908932,00:17
5,0.282006,0.237996,0.915937,00:17


[I 2024-05-24 22:17:45,930] Trial 0 finished with value: 0.917688250541687 and parameters: {'arch': 'resnet34', 'wd': 0.0002481040974867811, 'epochs': 6, 'bs': 32, 'drop': 0.2}. Best is trial 0 with value: 0.917688250541687.


epoch,train_loss,valid_loss,accuracy,time
0,0.788204,0.412232,0.862522,00:21


...

[I 2024-05-24 22:43:13,933] Trial 8 finished with value: 0.9238178730010986 and parameters: {'arch': 'resnet34', 'wd': 0.0008241925264876454, 'epochs': 13, 'bs': 64, 'drop': 0.4}. Best is trial 7 with value: 0.9553415179252625.


epoch,train_loss,valid_loss,accuracy,time
0,0.754382,0.306970,0.893170,00:10


epoch,train_loss,valid_loss,accuracy,time
0,0.449796,0.255285,0.911559,00:12
1,0.400212,0.207009,0.919440,00:12
2,0.325846,0.204704,0.919440,00:12
3,0.275052,0.171263,0.931699,00:12
4,0.235291,0.169300,0.935201,00:12
5,0.199903,0.156644,0.941331,00:12
6,0.197643,0.150987,0.939580,00:12
7,0.164178,0.148437,0.936077,00:12


[I 2024-05-24 22:45:05,289] Trial 9 finished with value: 0.9360770583152771 and parameters: {'arch': 'resnet18', 'wd': 1.753594952976443e-05, 'epochs': 8, 'bs': 32, 'drop': 0.2}. Best is trial 7 with value: 0.9553415179252625.


Accuracy: 0.9553415179252625
Best hyperparameters: {'arch': 'resnet50', 'wd': 3.6618192203924288e-06, 'epochs': 13, 'bs': 64, 'drop': 0.2}


# Hyperparameter Optimization with Optuna's Successive Halving Pruner

The addition of SuccessiveHalvingPruner helps to improve efficiency during optimization. By early stopping trials with low validation accuracy (potentially bad performers), SHP focuses resources on trials that are more likely to be good. This can significantly reduce the total training time, especially for computationally expensive models.

For n_trails = 10, accuracy scores and hyper-paramters are:

| Trial | Best Score | Architecture | Weight Decay | Epochs | Batch Size | Drop               |
|-------|------------|--------------|--------------|--------|------------|--------------------|
| 0     | 0.9623     | resnet50     | 2.057e-06    | 8      | 32         | 0.2888             |
| 1     | 0.9518     | resnet50     | 0.001421     | 7      | 32         | 0.2707             |
| 2     | 0.9807     | resnet34     | 3.744e-06    | 9      | 32         | 0.3918             |
| 3     | 0.9641     | resnet18     | 2.667e-05    | 7      | 64         | 0.3196             |
| 4     | 0.9711     | resnet34     | 0.005883     | 8      | 64         | 0.2000             |
| 5     | 0.9650     | resnet18     | 5.694e-06    | 6      | 64         | 0.2266             |
| 6     | 0.9737     | resnet18     | 1.813e-05    | 6      | 64         | 0.3732             |
| 7     | 0.9851     | resnet34     | 0.004016     | 13     | 32         | 0.2680             |
| 8     | 0.9667     | resnet50     | 0.008095     | 15     | 32         | 0.3013             |
| 9     | 0.9632     | resnet50     | 0.0006995    | 6      | 32         | 0.3595             |


**The best aacuracy score is 0.9851 with these hyperparameters:**

- Architecture: ResNet 34
- Weight Decay: 0.004016
- Epochs: 13
- Batch Size: 32
- Drop: 0.268

SHA identifies and eliminates underperforming configurations early on. This prevents further resource expenditure on trials unlikely to yield good results. Optuna's Successive Halving (SHA) pruner can be advantageous over a simple random search for hyperparameter optimization.

In [8]:
from fastai.vision.all import *
import optuna
import numpy as np
from optuna.pruners import SuccessiveHalvingPruner

# objective function for Optuna
def objective(trial):
    path = Path('/kaggle/input/brain-tumor-mri-classification-dataset/Brain_Tumor_MRI_Image_Dataset/Training')

    # hyperparameters
    arch = trial.suggest_categorical('arch', [resnet18, resnet34, resnet50])
    wd = trial.suggest_loguniform('wd', 1e-6, 1e-2)
    epochs = trial.suggest_int('epochs', 5, 15)
    bs = trial.suggest_categorical('bs', [32, 64])
    drop = trial.suggest_float('drop', 0.2, 0.4)

    #DataBlock
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       get_items=get_image_files,
                       get_y=parent_label,
                       splitter=RandomSplitter(valid_pct=0.2, seed=42))
    
    dls = dblock.dataloaders(path, bs=bs)
    
    # learner
    learn = vision_learner(dls, arch, metrics=[accuracy], wd=wd)
    
    # Train the model
    learn.fine_tune(epochs, base_lr=0.001, cbs=[MixedPrecision()])
    
    # validation accuracy
    accuracy_metric = float(learn.validate()[1])
    
    return accuracy_metric

# Optuna study with the Halving Pruner
study = optuna.create_study(direction='maximize', pruner=SuccessiveHalvingPruner())

# Optimize the objective function
study.optimize(objective, n_trials=10)

# best trial
trial = study.best_trial

print("Best Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-05-24 23:18:32,835] A new study created in memory with name: no-name-3b9d03b9-fb10-4d8b-834d-1b4f896e975e


epoch,train_loss,valid_loss,accuracy,time
0,0.599265,0.300957,0.899300,00:23


epoch,train_loss,valid_loss,accuracy,time
0,0.264046,0.187430,0.922942,00:28
1,0.177461,0.192091,0.932574,00:28
2,0.111791,0.161029,0.949212,00:28
3,0.053418,0.128679,0.950088,00:28
4,0.039403,0.142595,0.951839,00:28
5,0.016710,0.131251,0.958844,00:28
6,0.020159,0.112070,0.965849,00:28
7,0.008556,0.110309,0.963222,00:28


[I 2024-05-24 23:22:49,077] Trial 0 finished with value: 0.9623467326164246 and parameters: {'arch': <function resnet50 at 0x7a55d092f880>, 'wd': 2.056715875355261e-06, 'epochs': 8, 'bs': 32, 'drop': 0.2887557735470722}. Best is trial 0 with value: 0.9623467326164246.


epoch,train_loss,valid_loss,accuracy,time
0,0.638909,0.292574,0.915061,00:22


...

[I 2024-05-24 23:47:49,546] Trial 8 finished with value: 0.9667250514030457 and parameters: {'arch': <function resnet50 at 0x7a55d092f880>, 'wd': 0.008095083336439402, 'epochs': 15, 'bs': 32, 'drop': 0.30129588154296516}. Best is trial 7 with value: 0.9851138591766357.


epoch,train_loss,valid_loss,accuracy,time
0,0.643114,0.311658,0.901051,00:22


epoch,train_loss,valid_loss,accuracy,time
0,0.260481,0.190513,0.932574,00:27
1,0.164268,0.152902,0.950963,00:27
2,0.076222,0.153643,0.953590,00:27
3,0.045928,0.119837,0.966725,00:27
4,0.019451,0.116562,0.965849,00:27
5,0.019660,0.117123,0.964098,00:27


[I 2024-05-24 23:51:04,891] Trial 9 finished with value: 0.9632224440574646 and parameters: {'arch': <function resnet50 at 0x7a55d092f880>, 'wd': 0.0006994648882504557, 'epochs': 6, 'bs': 32, 'drop': 0.3595360164107316}. Best is trial 7 with value: 0.9851138591766357.


Best Accuracy: 0.9851138591766357
Best hyperparameters: {'arch': <function resnet34 at 0x7a55d092f6d0>, 'wd': 0.004016134981130656, 'epochs': 13, 'bs': 32, 'drop': 0.2679709587877348}
